In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [112]:
train_data_path = '../data/train.csv'
test_data_path = '../data/test.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

# print(train_data.info())
# print(test_data.info())


In [113]:
# 1. 给test data的survived 补零，和train data合并，用于特征工程处理
test_data['Survived'] = 0
train_test = train_data.append(test_data)
train_test.shape
# train_test.info()
# test_data.shape
# train_data.shape



D:\ProgramFiles\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(1309, 12)

In [114]:
# 2. ‘Embarked’：判断是否为空，否则以众数(mode().iloc[0])进行填充
if train_test['Embarked'].isnull().sum() != 0:
    train_test['Embarked'].fillna(train_test['Embarked'].mode().iloc[0], inplace=True)

# 对 ‘Embarked’这一列进行拆分，类似于onehot编码(get_dummies())    
emb = pd.get_dummies(train_test['Embarked'], prefix='Embarked')
train_test = pd.concat([train_test, emb], axis=1)



In [115]:
# 3. 'Sex'：无缺失值，直接onehot
sex = pd.get_dummies(train_test['Sex'], prefix='Sex')
train_test = pd.concat([train_test, sex], axis=1)


In [116]:
# 4. 'Name'：从名字中提取称呼和姓名，
name = train_test['Name'].str.extract(",([\S\s]{1,})\.([\S\s]{1,})[\(]{0,1}")
title = name[0]
name = name[1]
 
train_test['Title'] = title
train_test['RealName'] = name
    

In [117]:
# 5. 将 称呼 进行分列
title = pd.get_dummies(train_test['Title'], prefix='Title')
train_test = pd.concat([train_test, title], axis=1)



In [118]:
# 6. ‘Fare’：填充缺失值，按Pclass等级的均价来填
if train_test['Fare'].isnull().sum() != 0:
    train_test['Fare'] = train_test[['Fare']].fillna(train_test.groupby('Pclass').transform('mean'))

# 泰坦尼克号中有家庭团体票（分析Ticket号可以得到），所以需要将团体票分到每个人
group_ticket = train_test['Fare'].groupby(by=train_test['Ticket']).transform('count')
train_test['Fare'] = train_test['Fare'] / group_ticket
    

In [119]:
# 7. 给票价分级，再分列
def fare_category(fare):
        if fare <= 4:
            return 0
        elif fare <= 10:
            return 1
        elif fare <= 30:
            return 2
        elif fare <= 45:
            return 3
        else:
            return 4
train_test['Fare_Category'] = train_test['Fare'].map(fare_category)

fare_cat_dummies_df = pd.get_dummies(train_test['Fare_Category'],prefix=train_test[['Fare_Category']].columns[0])
train_test = pd.concat([train_test, fare_cat_dummies_df], axis=1)
train_test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 44 columns):
Age                                1046 non-null float64
Cabin                              295 non-null object
Embarked                           1309 non-null object
Fare                               1309 non-null float64
Name                               1309 non-null object
Parch                              1309 non-null int64
PassengerId                        1309 non-null int64
Pclass                             1309 non-null int64
Sex                                1309 non-null object
SibSp                              1309 non-null int64
Survived                           1309 non-null int64
Ticket                             1309 non-null object
Embarked_C                         1309 non-null uint8
Embarked_Q                         1309 non-null uint8
Embarked_S                         1309 non-null uint8
Sex_female                         1309 non-null uint8
Sex_ma

In [129]:
# 8. 分别得到Pclass各个等级的平均票价
p1_mean = train_test['Fare'].groupby(by=train_test['Pclass']).mean().get([1]).values[0]
p2_mean = train_test['Fare'].groupby(by=train_test['Pclass']).mean().get([2]).values[0]
p3_mean = train_test['Fare'].groupby(by=train_test['Pclass']).mean().get([3]).values[0]

# 9. 根据Pclass各个等级平均票价，再对票价分等级
from sklearn.preprocessing import LabelEncoder        
def pclass_fare_category(df, Pclass_1_mean_fare, Pclass_2_mean_fare, Pclass_3_mean_fare):
    if df['Pclass'] == 1:
        if df['Fare'] <= Pclass_1_mean_fare:
            return 'Pclass_1_Low_Fare'
        else:
            return 'Pclass_1_High_Fare'
    elif df['Pclass'] == 2:
        if df['Fare'] <= Pclass_2_mean_fare:
            return 'Pclass_2_Low_Fare'
        else:
            return 'Pclass_2_High_Fare'
    elif df['Pclass'] == 3:
        if df['Fare'] <= Pclass_3_mean_fare:
            return 'Pclass_3_Low_Fare'
        else:
            return 'Pclass_3_High_Fare'
train_test['pclass_fare_category'] = train_test.apply(pclass_fare_category, args=(p1_mean,p2_mean,p3_mean), axis=1)
p_fare = LabelEncoder()
p_fare.fit(np.array(['Pclass_1_Low_Fare','Pclass_1_High_Fare','Pclass_2_Low_Fare','Pclass_2_High_Fare','Pclass_3_Low_Fare','Pclass_3_High_Fare']))
train_test['pclass_fare_category'] = p_fare.transform(train_test['pclass_fare_category'])





In [143]:
# 10. 将 Parch 和 SibSp 合并构成一个新的特征
def family_size_category(family_size):
    if family_size <= 1:
        return 'Single'
    elif family_size <= 3:
        return 'SmallFamily'
    else:
        return 'LargeFamily'
train_test['Family_size'] = train_test['Parch'] + train_test['SibSp'] + 1
train_test['Family_size_category'] = train_test['Family_size'].map(family_size_category)
family = LabelEncoder()
family.fit(np.array(['Single','SmallFamily','LargeFamily']))
train_test['Family_size_category'] = family.transform(train_test['Family_size_category'])
family_dummies = pd.get_dummies(train_test['Family_size_category'], prefix='Family_size_category')
train_test = pd.concat([train_test, family_dummies], axis=1)




In [155]:
# 11. 填充年龄。
# 可根据title的平均年龄填充，或中和sex title pclass的平均填充
# 也可利用机器学习算法预测
# 此处采用机器学习算法：将age完整的项合并为train， 缺失的项合并为test
train_test[['Parch','SibSp','Family_size','Family_size_category','Family_size_category_0','Family_size_category_1','Family_size_category_2']].head(20)
missing_age_df = pd.DataFrame(train_test[['Title','Age','Sex', 'Parch',  'SibSp', 'Family_size', 'Family_size_category', 'Fare', 'Fare_Category', 'Pclass', 'Embarked']])
missing_age_df = pd.get_dummies(missing_age_df,columns=['Title', 'Family_size_category', 'Fare_Category', 'Sex', 'Pclass' ,'Embarked'])
missing_age_train = missing_age_df[missing_age_df['Age'].notnull()]
missing_age_test = missing_age_df[missing_age_df['Age'].isnull()]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046 entries, 0 to 415
Data columns (total 40 columns):
Age                                1046 non-null float64
Parch                              1046 non-null int64
SibSp                              1046 non-null int64
Family_size                        1046 non-null int64
Fare                               1046 non-null float64
Title_ Capt                        1046 non-null uint8
Title_ Col                         1046 non-null uint8
Title_ Don                         1046 non-null uint8
Title_ Dona                        1046 non-null uint8
Title_ Dr                          1046 non-null uint8
Title_ Jonkheer                    1046 non-null uint8
Title_ Lady                        1046 non-null uint8
Title_ Major                       1046 non-null uint8
Title_ Master                      1046 non-null uint8
Title_ Miss                        1046 non-null uint8
Title_ Mlle                        1046 non-null uint8
Title_ Mme

Age  Parch  SibSp  Family_size       Fare  Title_ Capt  Title_ Col  \
5   NaN      0      0            1   8.458300            0           0   
17  NaN      0      0            1  13.000000            0           0   
19  NaN      0      0            1   7.225000            0           0   
26  NaN      0      0            1   7.225000            0           0   
28  NaN      0      0            1   7.879200            0           0   
29  NaN      0      0            1   7.895800            0           0   
31  NaN      0      1            2  48.840267            0           0   
32  NaN      0      0            1   7.750000            0           0   
36  NaN      0      0            1   7.229200            0           0   
42  NaN      0      0            1   7.895800            0           0   
45  NaN      0      0            1   8.050000            0           0   
46  NaN      0      1            2   7.750000            0           0   
47  NaN      0      0            1   7.750000            0           0   
48  NaN      0      2            3   7.226400            0           0   
55  NaN      0      0            1  35.500000            0           0   
64  NaN      0      0            1  27.720800            0           0   
65  NaN      1      1            3   5.081933            0           0   
76  NaN      0      0            1   7.895800            0           0   
77  NaN      0      0            1   8.050000            0           0   
82  NaN      0      0            1   7.787500            0           0   

    Title_ Don  Title_ Dona  Title_ Dr  ...  Fare_Category_3  Fare_Category_4  \
5            0            0          0  ...                0                0   
17           0            0          0  ...                0                0   
19           0            0          0  ...                0                0   
26           0            0          0  ...                0                0   
28           0            0          0  ...                0                0   
29           0            0          0  ...                0                0   
31           0            0          0  ...                0                1   
32           0            0          0  ...                0                0   
36           0            0          0  ...                0                0   
42           0            0          0  ...                0                0   
45           0            0          0  ...                0                0   
46           0            0          0  ...                0                0   
47           0            0          0  ...                0                0   
48           0            0          0  ...                0                0   
55           0            0          0  ...                1                0   
64           0            0          0  ...                0                0   
65           0            0          0  ...                0                0   
76           0            0          0  ...                0                0   
77           0            0          0  ...                0                0   
82           0            0          0  ...                0                0   

    Sex_female  Sex_male  Pclass_1  Pclass_2  Pclass_3  Embarked_C  \
5            0         1         0         0         1           0   
17           0         1         0         1         0           0   
19           1         0         0         0         1           1   
26           0         1         0         0         1           1   
28           1         0         0         0         1           0   
29           0         1         0         0         1           0   
31           1         0         1         0         0           1   
32           1         0         0         0         1           0   
36           0         1         0         0         1           1   
42           0         1         0         0   

In [156]:
from sklearn import model_selection
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

# 弃掉不需要的列
def drop_col_not_req(df, cols):
    df.drop(cols, axis=1, inplace=True)

def fill_missing_age(missing_age_train, missing_age_test):
    # 构造训练集、测试集的x、y
    missing_age_X_train = missing_age_train.drop(['Age'], axis=1)
    missing_age_Y_train = missing_age_train['Age']
    missing_age_X_test = missing_age_test.drop(['Age'], axis=1)
        
    #模型1
    gbm_reg = ensemble.GradientBoostingRegressor(random_state=42)
    gbm_reg_param_grid = {'n_estimators': [2000], 'max_depth': [3],'learning_rate': [0.01], 'max_features': [3]}
    gbm_reg_grid = model_selection.GridSearchCV(gbm_reg, gbm_reg_param_grid, cv=10, n_jobs=25, verbose=1,  scoring='neg_mean_squared_error')
    gbm_reg_grid.fit(missing_age_X_train, missing_age_Y_train)
    print('Age feature Best GB Params:' + str(gbm_reg_grid.best_params_))
    print('Age feature Best GB Score:' + str(gbm_reg_grid.best_score_))
    print('GB Train Error for "Age" Feature Regressor:'+ str(gbm_reg_grid.score(missing_age_X_train, missing_age_Y_train)))
    missing_age_test['Age_GB'] = gbm_reg_grid.predict(missing_age_X_test)
    print(missing_age_test['Age_GB'][:4])
        
    #模型2
    lrf_reg = LinearRegression()
    lrf_reg_param_grid = {'fit_intercept': [True], 'normalize': [True]}
    lrf_reg_grid = model_selection.GridSearchCV(lrf_reg, lrf_reg_param_grid, cv=10, n_jobs=25, verbose=1, scoring='neg_mean_squared_error')
    lrf_reg_grid.fit(missing_age_X_train, missing_age_Y_train)
    print('Age feature Best LR Params:' + str(lrf_reg_grid.best_params_))
    print('Age feature Best LR Score:' + str(lrf_reg_grid.best_score_))
    print('LR Train Error for "Age" Feature Regressor' + str(lrf_reg_grid.score(missing_age_X_train, missing_age_Y_train)))
    missing_age_test['Age_LRF'] = lrf_reg_grid.predict(missing_age_X_test)
    print(missing_age_test['Age_LRF'][:4])
    
    #将两个模型预测后的均值作为最终预测结果
    print('shape1',missing_age_test['Age'].shape,missing_age_test[['Age_GB','Age_LRF']].mode(axis=1).shape)
    #missing_age_test['Age'] = missing_age_test[['Age_GB','Age_LRF']].mode(axis=1)
    missing_age_test['Age'] = np.mean([missing_age_test['Age_GB'],missing_age_test['Age_LRF']])
    print(missing_age_test['Age'][:4])
    drop_col_not_req(missing_age_test, ['Age_GB', 'Age_LRF'])
        

    return missing_age_test
# train_test.loc[(train_test.Age.isnull()), 'Age'] = \
fill_missing_age(missing_age_train,missing_age_test)


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Age feature Best GB Params:{'learning_rate': 0.01, 'max_depth': 3, 'max_features': 3, 'n_estimators': 2000}
Age feature Best GB Score:-112.63870941587804
GB Train Error for "Age" Feature Regressor:-94.75827545531382
5     33.184767
17    33.534863
19    32.993873
26    26.649466
Name: Age_GB, dtype: float64
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Age feature Best LR Params:{'fit_intercept': True, 'normalize': True}
Age feature Best LR Score:-7.6796439490873655e+28
LR Train Error for "Age" Feature Regressor-112.70331333829395
5     34.687500
17    33.890625
19    31.351562
26    26.875000
Name: Age_LRF, dtype: float64
shape1 (263,) (263, 2)
5     29.394189
17    29.394189
19    29.394189
26    29.394189
Name: Age, dtype: float64


[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   5 out of  10 | elapsed:    6.6s remaining:    6.6s
[Parallel(n_jobs=25)]: Done  10 out of  10 | elapsed:    9.5s finished
D:\ProgramFiles\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   5 out of  10 | elaps

Age  Parch  SibSp  Family_size       Fare  Title_ Capt  Title_ Col  \
5    29.394189      0      0            1   8.458300            0           0   
17   29.394189      0      0            1  13.000000            0           0   
19   29.394189      0      0            1   7.225000            0           0   
26   29.394189      0      0            1   7.225000            0           0   
28   29.394189      0      0            1   7.879200            0           0   
29   29.394189      0      0            1   7.895800            0           0   
31   29.394189      0      1            2  48.840267            0           0   
32   29.394189      0      0            1   7.750000            0           0   
36   29.394189      0      0            1   7.229200            0           0   
42   29.394189      0      0            1   7.895800            0           0   
45   29.394189      0      0            1   8.050000            0           0   
46   29.394189      0      1            2   7.750000            0           0   
47   29.394189      0      0            1   7.750000            0           0   
48   29.394189      0      2            3   7.226400            0           0   
55   29.394189      0      0            1  35.500000            0           0   
64   29.394189      0      0            1  27.720800            0           0   
65   29.394189      1      1            3   5.081933            0           0   
76   29.394189      0      0            1   7.895800            0           0   
77   29.394189      0      0            1   8.050000            0           0   
82   29.394189      0      0            1   7.787500            0           0   
87   29.394189      0      0            1   8.050000            0           0   
95   29.394189      0      0            1   8.050000            0           0   
101  29.394189      0      0            1   7.895800            0           0   
107  29.394189      0      0            1   7.775000            0           0   
109  29.394189      0      1            2   8.050000            0           0   
121  29.394189      0      0            1   8.050000            0           0   
126  29.394189      0      0            1   7.750000            0           0   
128  29.394189      1      1            3   7.452767            0           0   
140  29.394189      2      0            3   5.081933            0           0   
154  29.394189      0      0            1   7.312500            0           0   
..         ...    ...    ...          ...        ...          ...         ...   
268  29.394189      0      0            1   8.050000            0           0   
271  29.394189      0      0            1   7.750000            0           0   
273  29.394189      0      1            2   7.750000            0           0   
274  29.394189      0      0            1   7.225000            0           0   
282  29.394189      0      0            1   7.750000            0           0   
286  29.394189      0      0            1   7.250000            0           0   
288  29.394189      0      0            1   7.229200            0           0   
289  29.394189      0      0            1   8.050000            0           0   
290  29.394189      0      0            1  39.600000            0           0   
292  29.394189      0      0            1   7.229200            0           0   
297  29.394189      0      2            3   7.226400            0           0   
301  29.394189      0      0            1  15.045800            0           0   
304  29.394189      0      0            1   7.750000            0           0   
312  29.394189      0      0            1   7.575000            0           0   
332  29.394189      0      0            1   7.225000            0           0   
339  29.394189      0      0            1   7.229200            0           0   
342  29.394189      9      1           11   6.322727            0           0   
344  29.394189      1      1            3 